Author: Yun Xing


Task 1: set up the environment and document preprocessing: the documents were loaded into a dataframe

In [22]:
import sklearn as sk
import nltk
import pandas as pd
import numpy as np
import os,sys
#nltk.download('stopwords')
df = pd.DataFrame(columns=["directory","filename","content"])
for (root,dirs, files) in os.walk('mini_newsgroups', topdown=True):
    for name in files:
        try:
            path=os.path.join(root,name)
            #print(os.path.join(root,name))
            #Read the file and add it to the dataframe as a new row containing the directory, the file name and the content of the file
            data=open(path,'r',encoding='ISO-8859-1')
            df.loc[len(df.index)]=[root,name,data]
        except:
            # anything wrong, print them
            print("error:", root, name)
            None 
#df.describe()
#df.head()
#get the number of document N, this number is calculated as the total number of rows of the dataframe subtract by 1
#as the first row is a file automaticaly generated by the mac system and does not belong to the data
N=df.shape[0]-1
print('Document preprocessing complete')

Document preprocessing complete


Task 1.2 Data preprocessing: This included gettign content from each document and convert the content into a bag of terms for each document 

In [23]:
def get_content(data):
    body=[]
    lineNumber=0
    lines=data.readlines()
    error =0 
        
    # get the subject line and line number of the body text
    for line in lines:
        if line.startswith("Subject:"):
            #print(line)
            l=line.lstrip('Subject:')
            #print(l)
            body.append(l)
            #print(body)
        if line.startswith("Lines:"):
            #print(line)
            ln=line.lstrip('Lines:')
            try:
                lineNumber=int(ln)
                break
            except:
                error=1
            #print(lineNumber)
    #get the body content
    doc_body=""
    if(error==0 and lineNumber>0):
        for line in lines[-lineNumber:]:
            bl=line.strip()
            body.append(bl)
            #print(body)
        doc_body=' '.join(body)
        #print(doc_body)
    return doc_body, error

In [24]:
def get_terms(string):
    #remove punctuation from string
    #initializing punctuation
    punc='''+-!,''.\n?;:()[]^"{}<>||_\/=*#'''
    #removing all punctuation in the document
    for ele in string:
        if ele in punc:
            string=string.replace(ele,'')
    #print(string)
    tokens=string.split(' ')
    #print(tokens)          
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    tokens=[t.lower() for t in tokens]
    #print(tokens)
    tokens=[t for t in tokens if t not in stop_words]
    #print(tokens)
    from nltk.stem import PorterStemmer
    ps = PorterStemmer()
    terms = [ps.stem(t) for t in tokens]
    #print('\n',terms)
    return terms
# collect the terms for each document and combine them all into a global_terms list
d_terms=[]
global_terms=[]
for doc in df['content']:
    #print(doc)
    doc_content,error=get_content(doc)
    doc_terms=get_terms(doc_content)
    #put all doc_terms in one column ("d_terms")
    d_terms.append(doc_terms)
    #put all doc_terms in one large list
    global_terms.extend(doc_terms)
#print(d_terms)
print('bag of terms complete')

bag of terms complete


Task 2.1. Create the global terms dictionary with unique terms, assign each term a term_id (or feature_id), starting from 0. 

In [25]:
#removal duplicate terms in the global_terms list using set()
global_terms=list(set(global_terms))
#remove the first term which is an empty space
del global_terms[0]
#print(global_terms)
#put the unqiue terms in a dictionary and assign term_ID to each term
length=len(global_terms)
g_terms={}
for j in range(length):
    terms={j:global_terms[j]}
    g_terms.update(terms)
#print(g_terms)
print('global terms complete')

def get_key(val):
    for key, value in g_terms.items():
        if val==value:
            return key

global terms complete


Task 2.2. Calculate the DF for each term in the global terms dictionary.

In [26]:
#put d_terms in a dictionary
d_terms1={}
l=len(d_terms)
for m in range(l):
    value=d_terms[m]
    key=m
    pair={key:value}
    d_terms1.update(pair)

#calculate the DF for each term & save the results in a dictionary
dict_DF={}
for term_ID,term in g_terms.items():
    DF=0
    for doc in d_terms1.values():
        if term in doc:
            DF+=1
    DF=DF/N
    dict={term_ID:DF}
    dict_DF.update(dict)
#print(dict_DF)
print('DF complete!')

DF complete!


Task 2.3 Assign class ID to each document: the 20 mini_newsgroups were divided into 6 classes and each document was assigned a class_ID based on the directory it belongs. The class IDs were then saved to a list. 

In [27]:
class_labels={'comp.graphics':0,'comp.os.ms-windows.misc':0,'comp.sys.ibm.pc.hardware':0,
          'comp.sys.mac.hardware':0, 'comp.windows.x':0,'rec.autos':1, 'rec.motorcycles':1,
            'rec.sport.baseball':1, 'rec.sport.hockey':1,'sci.crypt':2, 'sci.electronics':2, 
              'sci.med':2, 'sci.space':2,'misc.forsale':3,'talk.politics.misc':4, 'talk.politics.guns':4,
              'talk.politics.mideast':4,'talk.religion.misc':5, 'alt.atheism':5,'soc.religion.christian':5}
def class_ID(string):
    class_ID=class_labels.get(string)
    return class_ID  


#assign class ID to each document 
c_ID=[]
for i in range(1,2001):
    dir=df['directory'][i]
    s_dir=dir.split('/')[1]
    c_ID.append(class_ID(s_dir))

print('assign class ID complete!')   

assign class ID complete!


Task 2.4 Generate trainign data: This included calculating the TF, IDF and TFIDF for each document and saved them to separate lists.

In [28]:
def get_key(val):
    for key, value in g_terms.items():
        if val==value:
            return key

def features(document):
    dict_TF={}
    dict_IDF={}
    dict_TFIDF={}
    ulist=list(set(document))
    ulist.remove('')
    for t in ulist:
        TF=document.count(t)
        key=get_key(t)
        DF=dict_DF.get(key)
        dict1={key:TF}
        IDF=np.log10(1/float(DF))
        TFIDF=TF*IDF
        dict2={key:IDF}
        dict3={key:TFIDF}
        dict_TF.update(dict1)
        dict_IDF.update(dict2)
        dict_TFIDF.update(dict3)
    return dict_TF, dict_IDF,dict_TFIDF

# calculate the TF,IDF and TFIDF for each document
doc_TF=[]
doc_IDF=[]
doc_TFIDF=[]

for j in range(len(d_terms)):
    doc1=d_terms[j]
    doc11_TF,doc11_IDF,doc11_TFIDF=features(doc1)
    doc_TF.append(doc11_TF)
    doc_IDF.append(doc11_IDF)
    doc_TFIDF.append(doc11_TFIDF)

print('features complete!')  

features complete!


Task 2.5 Test case design,load data to text files and verify the quality of the training data files: test case design was performed manually, the three features (TF,IDF and TFIDF) along with class_ID were written to 3 separate text files and lastly the quality score of each of the 3 training data files was calculated.

Test case design, below are three cases chosen from the newsgroup and manually-obtained feature data:
file:75895, talk.politics.mideast,class_ID:4, terms: lebanes (TF:4), isra (TF:6),resist(TF:2),bomb(TF:1)
file:70337, misc.forsale, class_ID: 3,terms: motorcycl (TF:2),inexpens (TF:1),want(TF:1)
file:15322, sci.crypt.,class_ID:2, terms:encrypt (TF:1),feal (TF:3),algorithm (TF:2)

In [79]:
#check if the TF & class_ID results match that obtained manually
#The indices for file:78895, 70337 and 15322 are 1,1841 and 714 on the doccument list
print('class ID for the three documents:' )
print(c_ID[1],c_ID[1841],c_ID[714])
#place all chosen terms in a list
test_terms=['lebanes','isra','resist','bomb','motorcycl','inexpens','want','encrypt','feal','algorithm']
term_ID=[]
for term in test_terms:
    ID=global_terms.index(term)
    term_ID.append(ID)
print('term_ID for the selected terms:',term_ID)
print('TF for "lebanes","isra","resist","bomb" in file 75895:')
print(doc_TF[1].get(35514),doc_TF[1].get(16380),doc_TF[1].get(33601),doc_TF[1].get(26284))
print('TF for "motorcycl","inexpens","want" in file 70337:')
print(doc_TF[1841].get(17978),doc_TF[1841].get(21346),doc_TF[1841].get(26923))
print('TF for "encrypt","feal","algorithm" in file 15322:')
print(doc_TF[714].get(18226),doc_TF[714].get(5486),doc_TF[714].get(25337))

print('Test cases results match results obtained manually!')

class ID for the three documents:
4 3 2
term_ID for the selected terms: [35514, 16380, 33601, 26284, 17978, 21346, 26923, 18226, 5486, 25337]
TF for "lebanes","isra","resist","bomb" in file 75895:
4 6 2 1
TF for "motorcycl","inexpens","want" in file 70337:
2 1 1
TF for "encrypt","feal","algorithm" in file 15322:
1 3 2
Test cases results match results obtained manually!


In [80]:

# opening the TF txt file in 'w' mode
file = open('TF.text', 'w+')
  
for i in range(1,len(c_ID)):
    s =""
    s += str(c_ID[i]) +" "
    fids = list(doc_TF[i].keys())
    fids.sort()
    for fid in fids:
        s += str(fid)+":"+str(doc_TF[i][fid])+" "
    file.write(s+'\n')
file.close()

# opening the IDF txt file in 'w' mode
file = open('IDF.text', 'w+')
  
for i in range(1,len(c_ID)):
    s =""
    s += str(c_ID[i]) +" "
    fids = list(doc_IDF[i].keys())
    fids.sort()
    for fid in fids:
        s += str(fid)+":"+str(doc_IDF[i][fid])+" "
    file.write(s+'\n')
file.close()

# opening the IDF txt file in 'w' mode
file = open('TFIDF.text', 'w+')
  
for i in range(1,len(c_ID)):
    s =""
    s += str(c_ID[i]) +" "
    fids = list(doc_TFIDF[i].keys())
    fids.sort()
    for fid in fids:
        s += str(fid)+":"+str(doc_TFIDF[i][fid])+" "
    file.write(s+'\n')
file.close()
print('write to files complete')


write to files complete


In [86]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import load_svmlight_file
import math

def evaluate(training_filename):
    feature_vectors, targets = load_svmlight_file(training_filename)
    clf = MultinomialNB()
    scores = cross_val_score(clf, feature_vectors, targets, cv=5, scoring='f1_macro')
    print("f1_macro: %0.2f (+/- %0.2f)" % (scores.mean(), 1.96*scores.std()/math.sqrt(5) ))

print('TF score:')
evaluate('TF.text')
print('\nIDF score:')
evaluate('IDF.text')
print('\nTFIDF score:')
evaluate('TFIDF.text')
print('\nProject completed!')


TF score:
f1_macro: 0.70 (+/- 0.04)

IDF score:
f1_macro: 0.74 (+/- 0.04)

TFIDF score:
f1_macro: 0.75 (+/- 0.04)

Project completed!
